In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
import importlib
from torch.utils.data import DataLoader

from config import Blip2Config
from dataset import Blip2Dataset
from tokenizer import Blip2Tokenizer

In [2]:
import config, dataset, tokenizer
importlib.reload(config)
importlib.reload(dataset)
importlib.reload(tokenizer)

from config import Blip2Config
from dataset import Blip2Dataset
from tokenizer import Blip2Tokenizer

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
config = Blip2Config()

In [5]:
blip2_tokenizer = Blip2Tokenizer(config, tokenizer)
config.vocab_size = blip2_tokenizer.n_vocab
train_dataset = Blip2Dataset(config, split="train", tokenizer=blip2_tokenizer.tokenize_text)
train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)

Skipping non-image file: image


In [ ]:
import copy
import timm
from transformers import BertConfig, BertModel

class BertMLPBlock(nn.Module):
    def __init__(self, intermediate, output):
        super().__init__()
        self.intermediate = intermediate
        self.output = output

    def forward(self, x):
        intermediate_output = self.intermediate(x)
        return self.output(intermediate_output, x)
    

class BertEncoderBlock(nn.Module):
    def __init__(self, bert_layer, bert_config, is_cross_attn=False):
        super().__init__()
        self.bert_config = bert_config
        self.is_cross_attn = is_cross_attn
        self.self_attn = bert_layer.attention
        self.mlp_img_transformer = BertMLPBlock(bert_layer.intermediate, bert_layer.output)
        self.mlp_text_transformer = BertMLPBlock(
                    copy.deepcopy(bert_layer.intermediate), 
                    copy.deepcopy(bert_layer.output)
                    )
        if is_cross_attn:
            self.cross_attn = nn.MultiheadAttention(embed_dim=self.bert_config.hidden_size, 
                                                    num_heads=self.bert_config.num_attention_heads, 
                                                    batch_first=True)
            self.cross_layer_norm = nn.LayerNorm(self.bert_config.hidden_size)
        
    def forward(self, query_embds, img_embds, text_embds, attn_mask):
        _, Qs, _ = query_embds.shape
        _, Ts, _ = text_embds.shape

        combined_embds = torch.concat((query_embds, text_embds), dim=1) # B, Qs + Ts, D

        self_attn_output = self.self_attn(combined_embds, attention_mask=attn_mask)[0]
        query_embds = combined_embds[:, :Qs]
        text_embds= combined_embds[:, Qs:]
        
        if self.is_cross_attn:
            hidden_states = self.cross_attn(query_embds, img_embds, img_embds)[0]
            query_embds = self.cross_layer_norm(query_embds + hidden_states)

        query_embds = self.mlp_img_transformer(query_embds)
        text_embds = self.mlp_text_transformer(text_embds)
        return query_embds, text_embds


class QTransformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.bert_cfg  = BertConfig.from_pretrained("bert-base-uncased")
        self.bert_model = BertModel.from_pretrained("bert-base-uncased", config = self.bert_cfg)
        
        self.encoder = nn.ModuleList()
        for i, bert_layer in enumerate(self.bert_model.encoder.layer):
            self.encoder.append(BertEncoderBlock(bert_layer, self.bert_cfg, i % 2 == 0))
        
        qs = config.num_queries
        ts = config.context_length
        combined_seq_len = qs + ts

        ####  STAGE 1: ITC, ITM, ITG Loss Masks ####
        # ITC Loss Mask
        itc_attn_mask = torch.zeros((combined_seq_len, combined_seq_len))
        itc_attn_mask[:qs, :qs] = 1
        itc_attn_mask[qs:, qs:] = 1

        # ITM Loss Mask
        itm_attn_mask = torch.ones((combined_seq_len, combined_seq_len))

        # ITG Loss Mask
        itg_attn_mask = torch.ones((combined_seq_len, combined_seq_len))
        itg_attn_mask[:qs, qs:] = 0
        itg_attn_mask[qs:, qs:] = torch.tril(itg_attn_mask[qs:, qs:], diagonal=0)

        self.register_buffer("itc_attn_mask", itc_attn_mask)
        self.register_buffer("itm_attn_mask", itm_attn_mask)
        self.register_buffer("itg_attn_mask", itg_attn_mask)

        ####  STAGE 2: ####
        # ITC Loss Mask will be same as stage 1 and reused for stage 2

    def forward(self, query_embds, img_embds, cls_text_embds, dec_text_embds, stage):

        itc_query_embds = query_embds.clone()
        itm_query_embds = query_embds.clone()
        itg_query_embds = query_embds.clone()

        itc_text_embds = cls_text_embds.clone()
        itm_text_embds = cls_text_embds.clone()
        itg_text_embds = dec_text_embds.clone()

        itc_query_embds, itc_text_embds, itm_query_embds, itm_text_embds, itg_query_embds, itg_text_embds = None, None, None, None, None, None

        for encoder in self.encoder:
            itc_query_embds, itc_text_embds = encoder(itc_query_embds, img_embds, itc_text_embds, self.itc_attn_mask)
            if stage == 1:
                itm_query_embds, itm_text_embds = encoder(itm_query_embds, img_embds, itm_text_embds, self.itm_attn_mask)
                itg_query_embds, itg_text_embds = encoder(itg_query_embds, img_embds, itg_text_embds, self.itg_attn_mask)
        return itc_query_embds, itc_text_embds, itm_query_embds, itm_text_embds, itg_query_embds, itg_text_embds
    

class QFormer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.q_transformer = QTransformer(config)
        self.learned_query = nn.Parameter(torch.randn(config.num_queries, config.embedding_dim))
        self.output_embedding  = nn.Embedding(config.vocab_size, config.embedding_dim)
        self.position_embedding = nn.Embedding(config.context_length, config.embedding_dim)

        position_ids = torch.arange(self.config.context_length).unsqueeze(0)
        self.register_buffer("position_ids", position_ids)

    def forward(self, image_embedding: torch.tensor, cls_tokens: torch.tensor, dec_tokens: torch.tensor, stage:int):
        B, S, E = image_embedding.shape
        learned_query = self.learned_query.unsqueeze(0).expand(B, -1, -1)

        cls_text_embeddings = self.output_embedding(cls_tokens) #(S,768)
        cls_text_embeddings = cls_text_embeddings + self.position_embedding(self.position_ids.expand(B, -1))
        dec_text_embeddings = self.output_embedding(dec_tokens) #(S,768)
        dec_text_embeddings = dec_text_embeddings + self.position_embedding(self.position_ids.expand(B, -1))

        itc_query_embds, itc_text_embds, itm_query_embds, itm_text_embds, itg_query_embds, itg_text_embds = self.q_transformer(
            learned_query, image_embedding, cls_text_embeddings, dec_text_embeddings, stage)

        if itg_text_embds is not None:
            itg_logits = itg_text_embds @ self.output_embedding.weight.T # (S,Vocab_size)
        else:
            itg_logits = None

        return itc_query_embds, itc_text_embds, itm_query_embds, itm_text_embds, itg_logits


class Blip2Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.image_encoder = timm.create_model('vit_tiny_patch16_224', pretrained=True)
        self.image_encoder.reset_classifier(0)

        for param in self.image_encoder.parameters():
            param.requires_grad = False

        self.image_proj = nn.Linear(config.img_embd_dim, config.embedding_dim)

        self.lm_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
        
        self.q_former = QFormer(config)
        self.z_proj = nn.Linear(config.embedding_dim, config.lm_embedding_dim)
    
    def forward(self, image:torch.tensor, cls_caption:torch.tensor, dec_caption:torch.tensor, stage:int):
        image_embedding = self.image_encoder.forward_features(image)  # [B, C, F]
        image_embedding = self.image_proj(image_embedding)

        if(stage == 1):
            itc_query_embds, itc_text_embds, itm_query_embds, itm_text_embds, itg_logits = self.q_former(image_embedding, cls_caption, dec_caption, stage)
            return itc_query_embds, itc_text_embds, itm_query_embds, itm_text_embds, itg_logits
        
        elif(stage == 2):
            cls_caption_dummy = torch.zeros_like(cls_caption)
            dec_caption_dummy = torch.zeros_like(dec_caption)
            itc_query_embds, itc_text_embds, itm_query_embds, itm_text_embds, itg_logits = self.q_former(image_embedding, 
                                cls_caption_dummy, dec_caption_dummy, stage)
            z = self.z_proj(itc_query_embds)  # [B, Qs, D]
            
            return itc_query_embds, itc_text_embds, itm_query_embds, itm_text_embds, itg_logits
        return self.q_former(image_embedding, cls_caption, dec_caption)

        

/usr/local/lib/python3.8/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.8/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://g

In [7]:
blip2_model = Blip2Model(config)
print(blip2_model)

Blip2Model(
  (image_encoder): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=192, out_features=576, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (norm): Identity()
          (proj): Linear(in_features=192, out_features=192, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=192, out_features=768, bias=True)
        

In [8]:
device = torch.device('cuda')
img, cls_caption, dec_caption = next(iter(train_dataloader))

img = img.to(device)
cls_caption = cls_caption.to(device)
dec_caption = dec_caption.to(device)

model = blip2_model.to(device)

# model.eval()
# with torch.no_grad():
#     itc_query_embds, itc_text_embds, itm_query_embds, itm_text_embds, itg_logits = model(img, cls_caption, dec_caption, 1)

In [9]:
class ITCLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, query_embds, text_embds):
        # query_embds: B, 32, d
        # text_embds: B, 77, d
        text_logit = text_embds[:, :1] # B, 1, d
        B, _, _ = text_logit.shape
        B, Qs, d = query_embds.shape 
        query_embds = query_embds.reshape(B * Qs, d)
        text_embds = text_logit.squeeze()
        logits = query_embds @ text_embds.T   # B*Qs,B
        logits = torch.max(logits.reshape(B,Qs,B),dim=1)[0] # B,B
        label = torch.arange(B,device=query_embds.device)
        return (F.cross_entropy(logits,label)+ F.cross_entropy(logits.T,label)) / 2, logits

class ITMLoss(nn.Module):
    def __init__(self, config):
        super().__init__()
        d = config.embedding_dim
        self.classification_layer = nn.Linear(d,2)
    
    def forward(self, query_embd, label):
        # query_embd --> (B,32,768)
        #label ->(B,1) B x [0/1]

        match_logit = self.classification_layer(query_embd) #(B,32,2)
        match_logit = match_logit.mean(dim=1)
        return F.cross_entropy(match_logit,label)


class ITGLoss(nn.Module):
    def __init__(self, pad_token_id):
        super().__init__()
        self.pad_token_id = pad_token_id

    def forward(self, itg_logits, label_token):
        #itg_logits -> B,S,vocab size
        #label_token -> B,S
        B, S, V = itg_logits.shape
        loss = F.cross_entropy(
            itg_logits.view(B * S, V),
            label_token.view(B * S),
            ignore_index=self.pad_token_id
        )
        return loss


In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)
num_epochs = 3

itc_loss_func = ITCLoss()
itm_loss_func = ITMLoss(config)
itg_loss_func = ITGLoss(blip2_tokenizer.pad_token_id)

itc_loss_func = itc_loss_func.to(device)
itm_loss_func = itm_loss_func.to(device)
itg_loss_func = itg_loss_func.to(device)

model = model.to(device)
model.train()

for epoch in range(num_epochs):
    iter = 0
    print(f"***************   Epoch {epoch + 1}  ***************")
    for img, cls_caption, dec_caption in train_dataloader:
        img = img.to(device)
        cls_caption = cls_caption.to(device)
        dec_caption = dec_caption.to(device)
        B, _, _, _ = img.shape

        itc_query_embds, itc_text_embds, itm_query_embds, itm_text_embds, itg_logits = model(img, cls_caption, dec_caption, 1)

        # ITC Loss
        itc_loss, itc_logits = itc_loss_func(itc_query_embds, itc_text_embds)

        # ITM Loss
        idx = torch.arange(B,device = device)
        itc_logits[idx,idx] = -1e9
        next_best_text_value , next_best_text_idx = torch.max(itc_logits,dim=1)
        mismatched_cls_caption = cls_caption[next_best_text_idx]
        mismatched_dec_caption = dec_caption[next_best_text_idx]

        _,_,mismatched_itm_query_embeds,_,_ = model(img, mismatched_cls_caption, mismatched_dec_caption,1)

        itm_query_embed_concatenated = torch.concat((itm_query_embds, mismatched_itm_query_embeds) ,dim=0 )
        itm_labels = torch.zeros(2 * B, dtype=torch.long).to(device)
        itm_labels[B:] = 1
        itm_loss = itm_loss_func(itm_query_embed_concatenated, itm_labels)

        # ITG Loss
        itg_labels = torch.concat((dec_caption[:, 1:], dec_caption[:, -1].unsqueeze(1)), dim=1)
        itg_loss = itg_loss_func(itg_logits, itg_labels)


        total_loss = itc_loss + itm_loss + itg_loss
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        print(f"Epoch {epoch + 1} : Iter [{iter} / {len(train_dataloader)}]")
        print(f"Total Loss: {total_loss.item()}")
        print(f"ITC Loss: {itc_loss}, ITM Loss: {itm_loss}, ITG Loss: {itg_loss}")
        print("" + "*" * 50)
        iter += 1



***************   Epoch 1  ***************
Epoch 1 : Iter [0 / 506]
Total Loss: 87.76075744628906
ITC Loss: 26.80521011352539, ITM Loss: 0.7197456359863281, ITG Loss: 60.235801696777344
**************************************************
Epoch 1 : Iter [1 / 506]
Total Loss: 60.467437744140625
ITC Loss: 13.219557762145996, ITM Loss: 0.7023423314094543, ITG Loss: 46.54553985595703
**************************************************
Epoch 1 : Iter [2 / 506]
Total Loss: 54.36392593383789
ITC Loss: 8.23693561553955, ITM Loss: 0.7711031436920166, ITG Loss: 45.35588836669922
**************************************************
Epoch 1 : Iter [3 / 506]
Total Loss: 48.50190734863281
ITC Loss: 5.196666717529297, ITM Loss: 0.7233944535255432, ITG Loss: 42.581844329833984
**************************************************
Epoch 1 : Iter [4 / 506]
Total Loss: 36.243194580078125
ITC Loss: 3.8224618434906006, ITM Loss: 0.6984116435050964, ITG Loss: 31.722320556640625
*************************************

KeyboardInterrupt: 